In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re

critical_keys1 = ['mod3', 'modp']
critical_keys2 = ['nc']
save = False

global_path = './outputs'

for c1 in critical_keys1:
    for c2 in critical_keys2:
        for c2_value in [True, False]:
            all_result_paths = glob.glob(global_path + '/mod*_exp/**/result.csv', recursive=True)
            all_result_paths = [p for p in all_result_paths if c1 in p]
            all_result_paths = [p for p in all_result_paths if (c2 in p)==c2_value]

            result_summary = {}
            n_layers = 6
            for result_path in all_result_paths:
                result = pd.read_csv(result_path)
                top_test_result = result['test_acc'].max()
                pe_type = 'NoPE' if 'nope' in result_path else 'Original'
                pattern = r"res=\[(.*?)\]"
                residual_list = re.findall(pattern, result_path)[0]
                residual_list = [int(num) for num in residual_list.split(", ")] if residual_list!='' else []
                # residual_list = [int(num) for num in residual_list.split(", ")]
                missing_rc = [i for i in range(n_layers) if i not in residual_list]
                missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


                if pe_type not in result_summary:
                    result_summary[pe_type] = {}
                if missing_rc_str not in result_summary[pe_type]:
                    result_summary[pe_type][missing_rc_str] = []
                result_summary[pe_type][missing_rc_str].append(top_test_result)
                if pe_type == 'Original' and missing_rc_str == '{5}':
                    print(result_path, pe_type, missing_rc_str, top_test_result)

            from ast import literal_eval
            mean_results = {}
            for pe_type in result_summary:
                cur_type_dict = result_summary[pe_type].copy()
                for missing_rc in cur_type_dict:
                    cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
                mean_results[pe_type] = cur_type_dict

            df = pd.DataFrame(mean_results)
            missing0 = ['{}']
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            display(df.loc[missing0,:].T)
            display(df.loc[missing1,:].T)
            display(df.loc[missing2,:].T)
            display(df.loc[missing3,:].T)
            display(df.loc[missing4,:].T)
            display(df.loc[missing5,:].T)
            num_seeds = df.values.min()


            from collections import OrderedDict
            final_results = OrderedDict()
            for pe_type in {'Original', 'NoPE'}:
                cur_type_dict = result_summary[pe_type].copy()
                final_dict = {
                    'min': cur_type_dict.copy(),
                    'max': cur_type_dict.copy(),
                    'avg.': cur_type_dict.copy(),
                }
                for missing_rc in cur_type_dict:
                    final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
                    final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
                    final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

                for k in final_dict:
                    final_results[pe_type+' ('+k+')'] = final_dict[k]

            df = pd.DataFrame(final_results)
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            r1 = df.loc[missing1,:].T
            r2 = df.loc[missing2,:].T
            r3 = df.loc[missing3,:].T
            r4 = df.loc[missing4,:].T
            r5 = df.loc[missing5,:].T
            r0 = df.loc[['{}'],:].T
            display(r0)
            display(r1)
            display(r2)
            display(r3)
            display(r4)
            display(r5)

            # write all 3 to the same csv file
            if save:
                all_r = pd.concat([r0, r1, r2, r3, r4, r5], axis=1)
                c2 = '_' + c2 if c2_value else ''
                
                all_r.to_csv(f'{c1}{c2}_{num_seeds}seeds.csv')



./outputs/mod3_nc_residual_exp/mod3_nc_sd240_T2403261540_res=[0, 1, 2, 3, 4]_ncTrue/mod3_nc_sd240_T2403261540_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 99.95
./outputs/mod3_nc_residual_exp/mod3_nc_sd241_T2403261928_res=[0, 1, 2, 3, 4]_ncTrue/mod3_nc_sd241_T2403261928_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 99.02
./outputs/mod3_nc_residual_exp/mod3_nc_sd242_T2403262316_res=[0, 1, 2, 3, 4]_ncTrue/mod3_nc_sd242_T2403262316_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 81.49
./outputs/mod3_nc_residual_exp/mod3_nc_sd243_T2403270301_res=[0, 1, 2, 3, 4]_ncTrue/mod3_nc_sd243_T2403270301_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 100.0
./outputs/mod3_nc_residual_exp/mod3_nc_sd244_T2403270643_res=[0, 1, 2, 3, 4]_ncTrue/mod3_nc_sd244_T2403270643_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 97.14


,{}
Original,5
NoPE,5


,{0},{1},{2},{3},{4},{5}
Original,5,5,5,5,5,5
NoPE,5,5,5,5,5,5


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original,5,5,5,5,5
NoPE,5,5,5,5,5


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original,5,5,5,5
NoPE,5,5,5,5


...

,{0},{1},{2},{3},{4},{5}
NoPE (min),99.990,100.0,99.330,100.0,100.0,100.0
NoPE (max),100.000,100.0,100.000,100.0,100.0,100.0
NoPE (avg.),99.998,100.0,99.866,100.0,100.0,100.0
Original (min),100.000,100.0,100.000,100.0,100.0,100.0
Original (max),100.000,100.0,100.000,100.0,100.0,100.0
Original (avg.),100.000,100.0,100.000,100.0,100.0,100.0


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),99.170,99.960,99.63,100.000,99.350
NoPE (max),100.000,100.000,100.00,100.000,100.000
NoPE (avg.),99.676,99.988,99.92,100.000,99.854
Original (min),100.000,100.000,99.96,99.910,100.000
Original (max),100.000,100.000,100.00,100.000,100.000
Original (avg.),100.000,100.000,99.99,99.982,100.000


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),99.170,99.130,99.330,100.000
NoPE (max),100.000,100.000,100.000,100.000
NoPE (avg.),99.734,99.826,99.862,100.000
Original (min),99.960,100.000,99.840,99.630
Original (max),100.000,100.000,100.000,100.000
Original (avg.),99.992,100.000,99.968,99.926


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),99.160,97.390,84.870
NoPE (max),99.990,100.000,99.890
NoPE (avg.),99.702,99.456,94.064
Original (min),93.500,80.800,99.210
Original (max),100.000,100.000,100.000
Original (avg.),98.634,96.160,99.832


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),54.400,36.410
NoPE (max),99.890,89.620
NoPE (avg.),83.668,63.036
Original (min),33.670,33.490
Original (max),98.630,97.390
Original (avg.),62.042,60.590
